In [ ]:
# Get data

from aocd import get_data
import math
import itertools
import networkx as nx
import matplotlib.pyplot as plt

# get data
data = get_data(day=10, year=2023)
data = data.split("\n")

data_h = len(data)
data_w = len(data[0])

# pad the data with extra ground round the outside
# (need this for part 2, and doesn't affect part 1)
data = ["." + line + "." for line in data]
data = ["." * (data_w+2)]+data+["." * (data_w + 2)]

#print(*data, sep = "\n")

In [ ]:

# The hard part is parsing the input into a Graph
# using networkx. This chunck does that, and then
# the next just finds all the nodes connected to S
# and prints the distance to the farthest one

# define the node index from x,y coordinates
def ni(x,y):
    return(str(x) + "," + str(y))

# empty graph
G = nx.Graph()
# iterate though the lines and the characters on each line
# the x and the y are the coordinates from the top left
# point. Note that duplicating pipes doesn't matter as
# the graph will just ignore edges that already exist
for y, line in enumerate(data):
    for x, char in enumerate(line):
        # go through the different pipe types in turn
        # and find the pipe coordinates they would
        # be connected to
        if char == "-":
            new_x1 = x-1
            new_y1 = y
            new_x2 = x+1
            new_y2 = y
            # but only connect the pipe if the adjacent
            # pipe is connecting the the correct direction
            if data[new_y1][new_x1] in ["-", "L", "F"]:
                G.add_edge(ni(x,y), ni(new_x1, new_y1))
            if data[new_y2][new_x2] in ["-", "J", "7"]:
                G.add_edge(ni(x,y), ni(new_x2, new_y2))
        # continue for the other pipe types...
        elif char == "|":
            new_x1 = x
            new_y1 = y-1
            new_x2 = x
            new_y2 = y+1
            if data[new_y1][new_x1] in ["|", "F", "7"]:
                G.add_edge(ni(x,y), ni(new_x1, new_y1))
            if data[new_y2][new_x2] in ["|", "L", "J"]:
                G.add_edge(ni(x,y), ni(new_x2, new_y2))
        elif char == "L":
            new_x1 = x
            new_y1 = y-1
            new_x2 = x+1
            new_y2 = y
            if data[new_y1][new_x1] in ["|", "F", "7"]:
                G.add_edge(ni(x,y), ni(new_x1, new_y1))
            if data[new_y2][new_x2] in ["-", "J", "7"]:
                G.add_edge(ni(x,y), ni(new_x2, new_y2))
        elif char == "J":
            new_x1 = x
            new_y1 = y-1
            new_x2 = x-1
            new_y2 = y
            if data[new_y1][new_x1] in ["|", "F", "7"]:
                G.add_edge(ni(x,y), ni(new_x1, new_y1))
            if data[new_y2][new_x2] in ["-", "L", "F"]:
                G.add_edge(ni(x,y), ni(new_x2, new_y2))
        elif char == "7":
            new_x1 = x-1
            new_y1 = y
            new_x2 = x
            new_y2 = y+1
            if data[new_y1][new_x1] in ["-", "F", "L"]:
                G.add_edge(ni(x,y), ni(new_x1, new_y1))
            if data[new_y2][new_x2] in ["|", "L", "J"]:
                G.add_edge(ni(x,y), ni(new_x2, new_y2))
        elif char == "F":
            new_x1 = x+1
            new_y1 = y
            new_x2 = x
            new_y2 = y+1
            if data[new_y1][new_x1] in ["-", "J", "7"]:
                G.add_edge(ni(x,y), ni(new_x1, new_y1))
            if data[new_y2][new_x2] in ["|", "J", "L"]:
                G.add_edge(ni(x,y), ni(new_x2, new_y2))
        # S also has to be connected
        elif char == "S":
            sx = x
            sy = y
            if data[y-1][x] in ["|", "F", "7"]:
                G.add_edge(ni(x,y), ni(x, y-1))
            if data[y+1][x] in ["|", "J", "L"]:
                G.add_edge(ni(x,y), ni(x, y+1))
            if data[y][x-1] in ["-", "F", "L"]:
                G.add_edge(ni(x,y), ni(x-1, y))
            if data[y][x+1] in ["-", "J", "7"]:
                G.add_edge(ni(x,y), ni(x+1, y))

#nx.draw(G, with_labels=True, font_weight='bold')
  

In [ ]:
  


print("Start coordinate:", sx,sy)
# check a cycle exists (even though we don't use it this time)
cycle_from_S = nx.find_cycle(G, source = ni(sx, sy))

# fina all the nodes which are connected to S and their distance from S
paths_from_S = nx.single_source_shortest_path(G, source = ni(sx, sy))
#print(paths_from_S)
# extract the distances and print
lengths_from_S = [len(path) for path in paths_from_S.values()]
print("Part 1:", max(lengths_from_S)-1)

In [ ]:
# Part 2

# same graph, but with intermediate points between each node
# so that there will be a line of nodes between adjacent
# parallel pipes. The intermediate points are all offset
# from existing coordinates by 0.5

# define the node index from x,y coordinates
# for part 2 this needs to make sure the half
# points are formatted correctly
def ni2(x,y):
    return("{:.1f}".format(x) + "," + "{:.1f}".format(y))

Gp2 = nx.Graph()
for y, line in enumerate(data):
    for x, char in enumerate(line):
        if char == "-":
            new_x1 = x-1
            new_y1 = y
            new_x2 = x+1
            new_y2 = y
            if data[new_y1][new_x1] in ["-", "L", "F", "S"]:
                Gp2.add_edge(ni2(x,y), ni2((x+new_x1)/2, (y+new_y1)/2))
            if data[new_y2][new_x2] in ["-", "J", "7", "S"]:
                Gp2.add_edge(ni2(x,y), ni2((x+new_x2)/2, (y+new_y2)/2))
        elif char == "|":
            new_x1 = x
            new_y1 = y-1
            new_x2 = x
            new_y2 = y+1
            if data[new_y1][new_x1] in ["|", "F", "7", "S"]:
                Gp2.add_edge(ni2(x,y), ni2((x+new_x1)/2, (y+new_y1)/2))
            if data[new_y2][new_x2] in ["|", "L", "J", "S"]:
                Gp2.add_edge(ni2(x,y), ni2((x+new_x2)/2, (y+new_y2)/2))
        elif char == "L":
            new_x1 = x
            new_y1 = y-1
            new_x2 = x+1
            new_y2 = y
            if data[new_y1][new_x1] in ["|", "F", "7", "S"]:
                Gp2.add_edge(ni2(x,y), ni2((x+new_x1)/2, (y+new_y1)/2))
            if data[new_y2][new_x2] in ["-", "J", "7", "S"]:
                Gp2.add_edge(ni2(x,y), ni2((x+new_x2)/2, (y+new_y2)/2))
        elif char == "J":
            new_x1 = x
            new_y1 = y-1
            new_x2 = x-1
            new_y2 = y
            if data[new_y1][new_x1] in ["|", "F", "7", "S"]:
                Gp2.add_edge(ni2(x,y), ni2((x+new_x1)/2, (y+new_y1)/2))
            if data[new_y2][new_x2] in ["-", "L", "F", "S"]:
                Gp2.add_edge(ni2(x,y), ni2((x+new_x2)/2, (y+new_y2)/2))
        elif char == "7":
            new_x1 = x-1
            new_y1 = y
            new_x2 = x
            new_y2 = y+1
            if data[new_y1][new_x1] in ["-", "F", "L", "S"]:
                Gp2.add_edge(ni2(x,y), ni2((x+new_x1)/2, (y+new_y1)/2))
            if data[new_y2][new_x2] in ["|", "L", "J", "S"]:
                Gp2.add_edge(ni2(x,y), ni2((x+new_x2)/2, (y+new_y2)/2))
        elif char == "F":
            new_x1 = x+1
            new_y1 = y
            new_x2 = x
            new_y2 = y+1
            if data[new_y1][new_x1] in ["-", "J", "7", "S"]:
                Gp2.add_edge(ni2(x,y), ni2((x+new_x1)/2, (y+new_y1)/2))
            if data[new_y2][new_x2] in ["|", "J", "L", "S"]:
                Gp2.add_edge(ni2(x,y), ni2((x+new_x2)/2, (y+new_y2)/2))
        elif char == "S":
            sx = x
            sy = y
            if data[y-1][x] in ["|", "F", "7"]:
                Gp2.add_edge(ni2(x,y), ni2(x, y-0.5))
            if data[y+1][x] in ["|", "J", "L"]:
                Gp2.add_edge(ni2(x,y), ni2(x, y+0.5))
            if data[y][x-1] in ["-", "F", "L"]:
                Gp2.add_edge(ni2(x,y), ni2(x-0.5, y))
            if data[y][x+1] in ["-", "J", "7"]:
                Gp2.add_edge(ni2(x,y), ni2(x+0.5, y))

#nx.draw(Gp2, with_labels=True, font_weight='bold')

# Check that the part 1 answer still works on this graph...
print(sx,sy)
cycle_from_Sp2 = nx.find_cycle(Gp2, source = ni2(sx, sy))
        
paths_from_Sp2 = nx.single_source_shortest_path(Gp2, source = ni2(sx, sy))
#print(paths_from_S)
lengths_from_Sp2 = [len(path) for path in paths_from_Sp2.values()]
print("Check part 1:", (max(lengths_from_Sp2)-1)//2)

#nx.draw(Gp2, with_labels=True, font_weight='bold')
  

In [ ]:

# then we construct a graph which has all the points
# and adjacent points, with each one connected to
# the four adjacent nodes

G_all = nx.Graph()
for y, line in enumerate(data):
    for x, char in enumerate(line):
        G_all.add_edge(ni2(x,y), ni2(x-0.5,y))
        G_all.add_edge(ni2(x,y), ni2(x+0.5,y))
        G_all.add_edge(ni2(x,y), ni2(x,y+0.5))
        G_all.add_edge(ni2(x,y), ni2(x,y-0.5))
        G_all.add_edge(ni2(x-0.5,y), ni2(x-0.5,y+0.5))
        G_all.add_edge(ni2(x-0.5,y), ni2(x-0.5,y-0.5))
        G_all.add_edge(ni2(x+0.5,y), ni2(x+0.5,y+0.5))
        G_all.add_edge(ni2(x+0.5,y), ni2(x+0.5,y-0.5))
        G_all.add_edge(ni2(x,y-0.5), ni2(x+0.5,y-0.5))
        G_all.add_edge(ni2(x,y-0.5), ni2(x-0.5,y-0.5))
        G_all.add_edge(ni2(x,y+0.5), ni2(x+0.5,y+0.5))
        G_all.add_edge(ni2(x,y+0.5), ni2(x-0.5,y+0.5))

#print(G_all.nodes())
#print(G_all.number_of_nodes())        

# we then remove all the nodes which are on the pipe cycle
# with S.  This separates the remaining nodes into an
# inside connected group and an outside connected group
for node in nx.node_connected_component(Gp2, ni2(sx,sy)):
    G_all.remove_node(node)
    
#nx.draw(G_all, with_labels=True, font_weight='bold')

# we then remove all the nodes which are connected to 
# the outside (or point 0,0 - which is not on the cycle
# because we padded out the input so 0,0 isn't on the
# original data)
for node in nx.node_connected_component(G_all, ni2(0,0)):
    G_all.remove_node(node)

# the remaining nodes are now the inside connected group
# but we don't want to count the intermediate nodes
# so we get a list with all the nodes which don't have a
# half way coordinate
centered_nodes = [node for node in G_all.nodes() if not ".5" in node]
# These are the nodes which we want!
print("Part 2:", len(centered_nodes))  

